<a href="https://colab.research.google.com/github/lilstarhunter/shrew_emesis/blob/main/shrew_emesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check that Runtime is set t GPU
# Install Dependencies
!pip install deeplabcut

     |████████████████████████████████| 541 kB 13.1 MB/s 
     |████████████████████████████████| 177 kB 35.7 MB/s 
     |████████████████████████████████| 296 kB 45.3 MB/s 
     |████████████████████████████████| 47.6 MB 37 kB/s 
     |████████████████████████████████| 9.8 MB 29.7 MB/s 
     |████████████████████████████████| 109 kB 43.4 MB/s 
     |████████████████████████████████| 29.2 MB 83 kB/s 
     |████████████████████████████████| 948 kB 50.5 MB/s 
     |████████████████████████████████| 352 kB 47.9 MB/s 
     |████████████████████████████████| 546 kB 50.1 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=df32af5e10c1e66001595e8192b270da995f73a6ce0275c2d3be041267c5b262
  Stored in directory: /root/.cache/pip/wheels/ce/e0/ee/a2b3c5caab3418c1ccd8c4de573d4cbe13315d7e8b0a55fbc2
Successfully built filterpy
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.16.2
    Uninstalling scikit-image-0.16.2:
    

In [2]:
# Use TensorFlow 1.x:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
# Link Google Drive files

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
import deeplabcut

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [5]:
import os
os.environ["DLClight"]="True"

In [6]:
#  Assign project variables
ProjectFolderName = 'vomitting-LMS-2021-10-19'
VideoType = 'mp4' 
videofile_path = ['/content/drive/My Drive/Colab Notebooks/shrew_emesis/'+ProjectFolderName+'/videos/'] 
path_config_file = '/content/drive/My Drive/Colab Notebooks/shrew_emesis/'+ProjectFolderName+'/config.yaml'

In [ ]:
# Create a training dataset
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

In [ ]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 1.15.2
Running on TPU  ['10.111.195.18:8470']


In [7]:
# Training - Iteration 0
shuffle = 1
deeplabcut.train_network(path_config_file, shuffle=shuffle, displayiters=100,saveiters=1000, maxiters=11000, allow_growth=True)

Selecting single-animal trainer


Config:
{'all_joints': [[0], [1], [2], [3], [4]],
 'all_joints_names': ['snout', 'mandible', 'back', 'abdomen', 'tailbase'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'batch_size': 1,
 'clahe': True,
 'claheratio': 0.1,
 'crop_pad': 0,
 'crop_sampling': 'hybrid',
 'crop_size': [400, 400],
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_vomittingOct19/vomitting_LMS95shuffle1.mat',
 'dataset_type': 'imgaug',
 'decay_steps': 30000,
 'deterministic': False,
 'display_iters': 1000,
 'edge': False,
 'emboss': {'alpha': [0.0, 1.0], 'embossratio': 0.1, 'strength': [0.5, 1.5]},
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'histeq': True,
 'histeqratio': 0.1,
 'init_weights': '/content/drive/My Drive/Colab '
                 'Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/dlc-models/iteration-0/vomittingOct19-trainset95shuffle1/train/snapshot-10000',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_hu

Batch Size is 1
Loading already trained DLC with backbone: resnet_50
Max_iters overwritten as 11000
Display_iters overwritten as 100
Save_iters overwritten as 1000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/dlc-models/iteration-0/vomittingOct19-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'imgaug', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfi

iteration: 10100 loss: 0.0126 lr: 0.02
iteration: 10200 loss: 0.0112 lr: 0.02
iteration: 10300 loss: 0.0104 lr: 0.02
iteration: 10400 loss: 0.0091 lr: 0.02
iteration: 10500 loss: 0.0093 lr: 0.02
iteration: 10600 loss: 0.0103 lr: 0.02
iteration: 10700 loss: 0.0093 lr: 0.02
iteration: 10800 loss: 0.0089 lr: 0.02
iteration: 10900 loss: 0.0089 lr: 0.02
iteration: 11000 loss: 0.0084 lr: 0.02


The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


Exception in thread Thread-12:
Traceback (most recent call last):
  File "/tensorflow-1.15.2/python3.7/tensorflow_core/python/client/session.py", line 1365, in _do_call
    return fn(*args)
  File "/tensorflow-1.15.2/python3.7/tensorflow_core/python/client/session.py", line 1350, in _run_fn
    target_list, run_metadata)
  File "/tensorflow-1.15.2/python3.7/tensorflow_core/python/client/session.py", line 1443, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledError: Enqueue operation was cancelled
	 [[{{node fifo_queue_enqueue}}]]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/core/train.py", line 82, in load_and_enqu

In [8]:
%matplotlib notebook
# Evaluate the model
shuffle=1
deeplabcut.evaluate_network(path_config_file,Shuffles=[shuffle], plotting=True)
#plot a few scoremaps:
# deeplabcut.extract_save_all_maps(path_config_file, shuffle=shuffle, Indices=[0,10,20])

NumExpr defaulting to 2 threads.


/content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/evaluation-results/  already exists!


Config:
{'all_joints': [[0], [1], [2], [3], [4]],
 'all_joints_names': ['snout', 'mandible', 'back', 'abdomen', 'tailbase'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_vomittingOct19/vomitting_LMS95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'mirror': False,
 'net_type': 'resnet_50',
 'num_joints': 5,
 'optimizer': 'sgd',
 'pairwise_huber_loss': True,
 'pairwise_predict': False,
 'partaffinityfield_predict': False,
 'regularize': False,
 'scoremap_dir': 'test',
 'shuffle': True,
 'snapshot_pre

/content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/evaluation-results/iteration-0/vomittingOct19-trainset95shuffle1  already exists!
Running  DLC_resnet50_vomittingOct19shuffle1_11000  with # of trainingiterations: 11000
Analyzing data...


535it [01:46,  5.02it/s]


Done and results stored for snapshot:  snapshot-11000
Results for 11000  training iterations: 95 1 train error: 7.37 pixels. Test error: 12.47  pixels.
With pcutoff of 0.6  train error: 5.52 pixels. Test error: 5.44 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
Plotting...


<IPython.core.display.Javascript object>

100%|██████████| 535/535 [02:51<00:00,  3.12it/s]


The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


In [17]:
# Analyze videos
videofile_path = ['/content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/videos/nicotine_emesis_10.19.21_shrew1_short.mp4'] 
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype=VideoType)

Config:
{'all_joints': [[0], [1], [2], [3], [4]],
 'all_joints_names': ['snout', 'mandible', 'back', 'abdomen', 'tailbase'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_vomittingOct19/vomitting_LMS95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'mirror': False,
 'net_type': 'resnet_50',
 'num_joints': 5,
 'optimizer': 'sgd',
 'pairwise_huber_loss': True,
 'pairwise_predict': False,
 'partaffinityfield_predict': False,
 'regularize': False,
 'scoremap_dir': 'test',
 'shuffle': True,
 'snapshot_pre

Using snapshot-11000 for model /content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/dlc-models/iteration-0/vomittingOct19-trainset95shuffle1
Starting to analyze %  /content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/videos/nicotine_emesis_10.19.21_shrew1_short.mp4
/content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/videos  already exists!
Loading  /content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/videos/nicotine_emesis_10.19.21_shrew1_short.mp4
Duration of video [s]:  29.66 , recorded with  59.94 fps!
Overall # of frames:  1778  found with (before cropping) frame dimensions:  750 452
Starting to extract posture


1785it [01:38, 18.19it/s]

Saving results in /content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/videos...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_resnet50_vomittingOct19shuffle1_11000'

In [18]:
videofile_path = ['/content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/videos/nicotine_emesis_10.19.21_shrew1_short.mp4'] 
deeplabcut.create_labeled_video(path_config_file,videofile_path, videotype="mp4")

/content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/videos  already exists!
Starting to process video: /content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/videos/nicotine_emesis_10.19.21_shrew1_short.mp4
Loading /content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/videos/nicotine_emesis_10.19.21_shrew1_short.mp4 and data.
Duration of video [s]: 29.66, recorded with 59.94 fps!
Overall # of frames: 1778 with cropped frame dimensions: 750 452
Generating frames and creating video.


 98%|█████████▊| 1750/1778 [00:10<00:00, 176.69it/s]

Error: %s axis 2 is out of bounds for array of dimension 0


 99%|█████████▊| 1754/1778 [00:10<00:00, 174.25it/s]


AxisError: ignored

In [ ]:
## Assess pose estimation (may need to add more data and re-train)
deeplabcut.create_video_with_all_detections(path_config_file, Specific_videofile)

In [ ]:
#Check and edit:
numAnimals = 1 #how many animals do you expect to find?
tracktype= 'ellipse' #box, skeleton, ellipse -- ellipse is recommended.

#Optional: imagine you tracked a point that is not useful for assembly, 
#like a tail tip that is far from the body, consider dropping it for this step (it's still used later)!
#To drop it, uncomment the next line TWO lines and add your parts(s):
bodypart= 'midchest, midhind'
deeplabcut.convert_detections2tracklets(path_config_file, videofile_path, videotype=VideoType, 
                                        shuffle=shuffle, track_method=tracktype, overwrite=True, ignore_bodyparts=[bodypart])

#OR don't drop, just click RUN:
# deeplabcut.convert_detections2tracklets(path_config_file, videofile_path, videotype=VideoType, 
                                        # shuffle=shuffle, track_method=tracktype, overwrite=True)

deeplabcut.stitch_tracklets(path_config_file, videofile_path, track_method=tracktype, n_tracks=numAnimals)

In [ ]:
# filter the data to remove any small jitter
deeplabcut.filterpredictions(path_config_file, 
                                 videofile_path, 
                                 videotype=VideoType)
                                #  track_method = tracktype)

Analyzing all the videos in the directory...
Filtering with median model /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0445.MP4
Saving filtered csv poses!
Filtering with median model /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0444.MP4
Saving filtered csv poses!


In [ ]:
#Create plot trajectory
deeplabcut.plot_trajectories(path_config_file, videofile_path, videotype=VideoType)

Analyzing all the videos in the directory...
Loading  /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0444.MP4 and data.
/content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/plot-poses/MVI_0444  already exists!
Loading  /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0445.MP4 and data.
/content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/plot-poses/MVI_0445  already exists!
Plots created! Please check the directory "plot-poses" within the video directory


In [ ]:
deeplabcut.create_labeled_video(path_config_file,
                                videofile_path, 
                                shuffle=shuffle, 
                                # color_by="individual",
                                videotype=VideoType, 
                                save_frames=False,
                                filtered=True) 
                                # track_method = tracktype)

Analyzing all the videos in the directory...
/content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos  already exists!
/content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos  already exists!
Starting to process video: /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0444.MP4
Starting to process video: /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0445.MP4
Loading /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0445.MP4 and data.
Loading /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0444.MP4 and data.
Duration of video [s]: 78.08, recorded with 29.97 fps!
Overall # of frames: 2340 with cropped frame dimensions: 1920 1080
Generating frames and creating video.


  1%|          | 21/2340 [00:00<01:19, 29.28it/s]

Duration of video [s]: 72.07, recorded with 29.97 fps!
Overall # of frames: 2160 with cropped frame dimensions: 1920 1080
Generating frames and creating video.


100%|██████████| 2340/2340 [02:22<00:00, 16.46it/s]
